# 앞으로 객체 크롤링 진행햐아 하는 제품

In [1]:
import os
import re
import pandas as pd
import pickle
import collections
import numpy as np
import math
from ast import literal_eval
from time import gmtime, strftime
import re
import time
from bs4 import BeautifulSoup
from tqdm import trange, tqdm_notebook
from tqdm.auto import tqdm

# Scrapping
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

# Error Handling
import socket
import urllib3
import urllib.request
from urllib.request import urlopen
from urllib.parse import quote_plus
from urllib.request import urlretrieve
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
import warnings
warnings.filterwarnings('ignore')

from collections import defaultdict

In [8]:
need_to_crawl = []

In [ ]:
df_brd_w_prd_v3 = pd.read_csv("df_brd_w_prd_v3.csv")
df_brd_w_prd_v3 = df_brd_w_prd_v3.iloc[:, 1:]

In [3]:
v4_update = pd.read_csv("v4_update.csv")
v4_update = v4_update.iloc[:, 1:]

In [5]:
lst_v4_prd_link = list(v4_update['제품 link'])

In [ ]:
#현재까지 크롤링 완료되어 한꺼번에 붙인 df
df_glowpick_prd_info = pd.read_csv("df_glowpick_prd_info.csv")
df_glowpick_prd_info = df_glowpick_prd_info.iloc[: 1:]

In [7]:
lst_df_glowpick_prd_link = list(df_glowpick_prd_info['제품 link'])

NameError: name 'df_glowpick_prd_info' is not defined

In [ ]:
for i in range(len(df_brd_w_prd_v3)):
    for j in list(df_brd_w_prd_v3.loc[i, 'product_link']):
        if j not in lst_df_glowpick_prd_link and j not in lst_v4_prd_link:
            need_to_crawl.append(j)

---
# 이미지 src만 추가로 받아와야 하는 상품 img src만 받아오기

In [6]:
#현재까지 크롤링 완료되어 한꺼번에 붙인 df
df_glowpick_prd_info_v1 = pd.read_csv("df_glowpick_prd_info_v1.csv")
df_glowpick_prd_info_v1 = df_glowpick_prd_info_v1.iloc[:, 1:]

In [7]:
def get_img_src(prd_link):
    
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    print(userAgent)

    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    options.add_argument(f'user-agent={userAgent}')
    wd = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

    wd.get(prd_link)
    time.sleep(5)
    wd.maximize_window()
    
    socket.setdefaulttimeout(30)
        
    try:
        
        wd.find_element_by_xpath('//*[@id=\"default-layout\"]/div/div[1]/span/div/div[2]/div[2]/button[2]').click()
        time.sleep(5)

        html = wd.page_source
        bs = BeautifulSoup(html, 'html.parser')
        
        time.sleep(3)
        img_src = wd.find_element_by_xpath("/html/body/div/div/div/div/main/div/section/div[1]/div/div/img").get_attribute('src')
        img_src = re.sub("w=[0-9]*", "w=500", img_src)
        wd.close()
        
        return img_src

    except Exception as e:
        exception.append(prd_link)
        print(e)

In [41]:
len(df_glowpick_prd_info_v1.loc[0, 'img_src'])

89

In [35]:
np.isnan(df_glowpick_prd_info_v1['img_src'].astype(np.float))

ValueError: could not convert string to float: 'https://dn5hzapyfrpio.cloudfront.net/home/glowmee/upload/20180319/1521441369802.png?w=500'

In [34]:
df_glowpick_prd_info_v1['img_src'] = df_glowpick_prd_info_v1['img_src'].astype(float)

ValueError: could not convert string to float: 'https://dn5hzapyfrpio.cloudfront.net/home/glowmee/upload/20180319/1521441369802.png?w=500'

In [33]:
df_glowpick_prd_info_v1.loc[np.isnan(df_glowpick_prd_info_v1['브랜드명'])]

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [29]:
#df_glowpick_prd_info는 지금까지 크롤링 완료된 glowpick 상품 df

for i in tqdm(list(df_glowpick_prd_info_v1.index)):
    try:
        if np.isnan(df_glowpick_prd_info_v1.loc[i, 'img_src']):
            prd_link = df_glowpick_prd_info_v1[i, '제품 link']
            img_src = get_img_src(prd_link)
            print('img src scraping complete')
            df_glowpick_prd_info_v1[i, 'img_src'] = img_src        
    except:
        pass

  0%|          | 0/17559 [00:00<?, ?it/s]

In [28]:
df_glowpick_prd_info_v1

,브랜드 link,브랜드명,브랜드 id,제품 link,제품 id,제품명,제품 용량,제품 가격,유사 제품군 랭킹(int),유사 제품군 랭킹(text),...,제품 카테고리,리뷰,별점,리뷰 일자,사용자 리뷰 개수,사용자 피부 타입,사용자 나이,사용자 성별,사용자 이름,img_src
0,NaN,가가알로에,NaN,https://www.glowpick.com/products/104713,104713,라율 나이트 케어 너리싱 젤,100g,"27,000원",472위,수분크림,...,"['스킨케어', '크림', '수분크림']","['발림성 좋고 촉촉함도 오래가며 많은애착이 갑니다', '저는 악건성 피부라 아주 ...","['4', '4', '4', '5', '5', '5', '5']","['4개월전', '2021.02.14', '2021.01.23', '2020.11....","['리뷰 36', '리뷰 71', '리뷰 59', '리뷰 97', '리뷰 23', ...","['복합성', '건성', '복합성', '건성', '지성', '민감성', '복합성']","['50세', '33세', '24세', '19세', '27세', '27세', '32세']","['여', '여', '남', '여', '여', '여', '여']","['땡또이', '소롱둥', '글로우맨이좋아', '[신유진]', '예우니95', 'ㄴ...",https://dn5hzapyfrpio.cloudfront.net/home/glow...
1,NaN,가디스가든,NaN,https://www.glowpick.com/products/96142,96142,유기농 드림 리페어 선 리페어 나이트 크림,28g,가격미정,120위,나이트크림,...,"['스킨케어', '크림', '나이트크림']",['잠들기전 꼭바르고자고 아침에 일어나면 촉촉하고 기분 좋아요'],['4'],['4개월전'],['리뷰 36'],['복합성'],['50세'],['여'],['땡또이'],https://dn5hzapyfrpio.cloudfront.net/home/glow...
2,NaN,가디스가든,NaN,https://www.glowpick.com/products/96141,96141,유기농 데이 언던 선 리페어 세럼,30ml,가격미정,NaN,NaN,...,"['스킨케어', '에센스/세럼', '진정에센스']",[],[],[],[],[],[],[],[],https://dn5hzapyfrpio.cloudfront.net/home/glow...
3,NaN,가디스가든,NaN,https://www.glowpick.com/products/96138,96138,천연 미네랄 선스크린 립 밤 라즈베리 맛,4g,가격미정,NaN,NaN,...,"['립메이크업', '립케어', '립밤']",[],[],[],[],[],[],[],[],https://dn5hzapyfrpio.cloudfront.net/home/glow...
4,NaN,가디스가든,NaN,https://www.glowpick.com/products/96133,96133,유기농 브라이트 아이즈 퍼밍 아이 크림,14g,가격미정,NaN,NaN,...,"['스킨케어', '크림', '아이크림']",[],[],[],[],[],[],[],[],https://dn5hzapyfrpio.cloudfront.net/home/glow...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17554,NaN,렌즈미,NaN,https://www.glowpick.com/products/100468,100468,열공STEP.3 그레이,2p,"10,000원",187위,그레이렌즈,...,"['렌즈', '컬러렌즈', '그레이렌즈']","['색이쁘고 가격저렴하지만 착용감 쓸애기', '진짜.. 너무 당황스러웠어요 내 눈 ...","['2', '5', '3', '5', '4', '5']","['2018.08.23', '2018.03.17', '2018.02.05', '20...","['리뷰 247', '리뷰 424', '리뷰 154', '리뷰 5', '리뷰 54'...","['지성', '건성', '민감성', '지성', '복합성', '민감성']","['23세', '21세', '25세', '18세', '24세', '24세']","['여', '여', '여', '여', '여', '여']","['안킬로사우루스', '10대에오', '리마킴', '목련7', '짓엉', '꿍쓰']",NaN
17555,NaN,렌즈미,NaN,https://www.glowpick.com/products/137509,137509,악마 홀로리스 원데이 진저브라운,30ea,"45,000원",225위,브라운렌즈,...,"['렌즈', '컬러렌즈', '브라운렌즈']","['제가 직경 너무 큰거 안좋아해서 자연스러운거 찾고 있었는데, 홀로리스 원데이가 ...","['5', '4', '3', '4']","['3개월전', '5개월전', '2021.04.01', '2021.02.20']","['리뷰 151', '리뷰 262', '리뷰 86', '리뷰 144']","['복합성', '민감성', '중성', '민감성']","['26세', '27세', '26세', '29세']","['여', '여', '여', '여']","['안녕미쯔', 'ㅊㄴ', '풀퓰', '호롤룰루야']",NaN
17556,NaN,렌즈미,NaN,https://www.glowpick.com/products/137262,137262,메이크오버 원데이 베이블리 브라운,4ea,"9,000원",227위,브라운렌즈,...,"['렌즈', '컬러렌즈', '브라운렌즈']",['너무 튀지 않는 그래픽 직경이라 자연스럽게 눈동자를 키워줘요! 직경 자체도 미디...,"['5', '4', '5', '3']","['3개월전', '2021.04.03', '2020.11.17', '2020.07....","['리뷰 8', '리뷰 150', '리뷰 109', '리뷰 281']","['민감성', '지성', '복합성', '민감성']","['23세', '23세', '30세', '27세']","['여', '여', '여', '여']","['v3t2tvcr', '하누리얌', 'woo__zzang', '안잘래']",NaN
17557,NaN,렌즈미,NaN,https://www.glowpick.com/products/72031,72031,본 내츄럴핏 바비 브라운,2p,"120,000원",228위,브라운렌즈,...,"['렌즈', '컬러렌즈', '브라운렌즈']","['이름처럼 매우 내추럴 합니당', '비싼값합니당 엄마 사랑햐♥♥♥2p를3개나 사줬...","['4', '5', '5', '3']","['2018.08.17', '2016.12.14', '2016.08.08', '20...","['리뷰 34', '리뷰 7', '리뷰 71', '리뷰 269']","['복합성', '복합성', '지성', '민감성']","['20세', '22세', '20세', '30세']","['여', '여', '여', '여']","['강다니엘팬', '너예쁘4가지', '힣힣핳핳', '위스덤']",NaN
